### CFB Scrape

In [1]:
import requests, pandas as pd
from bs4 import BeautifulSoup as soup
def extract_rankings(source):
   return [{"Rank":i.select_one('div.wrapper .rank-column .other').get_text(strip=True),
       "Team":i.select_one('.team').get_text(strip=True),
       "Total":i.select_one('.total').get_text(strip=True).split()[0],
       "5-Stars":i.select_one('.star-commits-list li:nth-of-type(1) div').get_text(strip=True),
       "4-Stars":i.select_one('.star-commits-list li:nth-of-type(2) div').get_text(strip=True),
       "3-Stars":i.select_one('.star-commits-list li:nth-of-type(3) div').get_text(strip=True),
       "Ave":i.select_one('.avg').get_text(strip=True),
       "Points":i.select_one('.points').get_text(strip=True),
    }
    for i in soup(source, 'html.parser').select("""li.rankings-page__list-item""")]

def year_rankings(year):
   page, results = 1, []
   vals = extract_rankings(requests.get(f'https://247sports.com/Season/{year}-Football/CompositeTeamRankings/?ViewPath=~%2FViews%2FSkyNet%2FInstitutionRanking%2F_SimpleSetForSeason.ascx&Page={page}', headers={'User-Agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.77 Mobile Safari/537.36'}).text)
   while vals:
      results.extend(vals)
      page += 1
      vals = extract_rankings(requests.get(f'https://247sports.com/Season/{year}-Football/CompositeTeamRankings/?ViewPath=~%2FViews%2FSkyNet%2FInstitutionRanking%2F_SimpleSetForSeason.ascx&Page={page}', headers={'User-Agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.77 Mobile Safari/537.36'}).text)
   return results

results = {y:year_rankings(y) for y in range(2000, 2025)}
df = pd.DataFrame([{'Year':a, **i} for a, b in results.items() for i in b])

In [2]:
with pd.ExcelWriter('U:\Administrative\Personal\cfb\cfb_rank.xlsx') as writer:
    df.to_excel(writer, sheet_name='cfb_recruiting')